# init

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io.arff import loadarff
from pymfe.mfe import MFE
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn import svm
import sklearn.metrics as skm
from sklearn import metrics
import pdb
import random

stdTable = "./data/recommender/std-table.csv"
path_arff = "./data/datasets/arff/"
path_index = "./data/index/arff/"
# files_arff = os.listdir(path_arff)

path = "./data/experimento/datasets/selected/cleaned/"
tablePath = savePath = "./data/recommender/meta-features.csv"

metaTablePath = "./metafeatures/meta-table.csv"

ccTable = "./metafeatures/meta-table-CC.csv"
accTable = "./metafeatures/meta-table-ACC.csv"
smmTable = "./metafeatures/meta-table-SMM.csv"
hdyTable = "./metafeatures/meta-table-HDy.csv"
dysTable = "./metafeatures/meta-table-DyS.csv"
sordTable = "./metafeatures/meta-table-SORD.csv"
msTable = "./metafeatures/meta-table-MS.csv"
# ms2Table = "./metafeatures/meta-table-MS2.csv"
maxTable = "./metafeatures/meta-table-MAX.csv"
xTable = "./metafeatures/meta-table-X.csv"
t50Table = "./metafeatures/meta-table-T50.csv"

algList = ["CC","ACC","SMM","HDy","DyS","SORD","MS","MAX","X","T50"]
pathList = [ccTable, accTable, smmTable, hdyTable, dysTable, sordTable, msTable, maxTable, xTable, t50Table]

datasets_path = './datasets/'
files = os.listdir(datasets_path)

# preprocess

In [ ]:
meta_table = pd.read_csv(metaTablePath)
arr_table = meta_table.drop(columns=meta_table.columns.tolist()[110::-1])
meta_table.drop(columns=meta_table.columns.tolist()[111::], inplace=True)
X = meta_table.values

In [ ]:
# alg_list = ['arr_PACC', 'arr_PCC', 'arr_MS2']
# arr_table.drop(columns=alg_list, inplace=True) 
arr_table

In [ ]:
instance_len = len(X)
knn = NearestNeighbors(n_neighbors=11)
ranking_table = pd.DataFrame(columns=arr_table.columns)

for i in range(0, instance_len):
   X_train = np.delete(X, i, 0)
   X_test = [X[i]]

   knn.fit(X_train)
   neighbors = knn.kneighbors(X=X_test)
   
   dt = 0
   for dist in neighbors[0].tolist()[0]:
      if dist == 0.0: ################################## GAMBIARRA ABSURDA -> CORRIGIR DEPOIS
         dist = 0.0000000000000001
      dt += (1 / dist)

   weight_list = []
   for dist in neighbors[0].tolist()[0]:
      if dist == 0.0: ################################## GAMBIARRA ABSURDA -> CORRIGIR DEPOIS
         dist = 0.0000000000000001
      weight_list.append((1 / dist) / dt)


   arr_list = [0] * len(arr_table.columns)
   weight_list_index = 0
   for neighbor in neighbors[1].tolist()[0]:

      arr_list_index = 0
      for arr in arr_table.loc[neighbor].tolist():
         arr_list[arr_list_index] += weight_list[weight_list_index] * arr
         arr_list_index += 1
      
      weight_list_index += 1
   
   ranking_table.loc[len(ranking_table.index)] = arr_list

ranking_table

# experiment

## Primeiro algoritmo recomendado está presente no conjunto dos ideias (por Top N)

In [ ]:
top = 3

hit = 0
for i in range(0, len(arr_table)):
    true_dict = {}
    j = 0
    true_arr_values = arr_table.loc[i].tolist()
    for column in arr_table.columns.tolist():
        true_dict[column] = true_arr_values[j]
        j += 1
    true_dict = sorted(true_dict.items(), key=lambda x:x[1], reverse=True)


    pred_dict = {}
    j = 0
    pred_arr_values = ranking_table.loc[i].tolist()
    for column in ranking_table.columns.tolist():
        pred_dict[column] = pred_arr_values[j]
        j += 1
    pred_dict = sorted(pred_dict.items(), key=lambda x:x[1], reverse=True)

    for top_index in range(0, top):
        if pred_dict[0][0] == true_dict[top_index][0]:
            hit += 1
            break

acc = hit / len(arr_table)

print('ACURÁCIA: ' + str(acc))

## Hit Rate

In [ ]:
y_axis = []
x_axis = []

for top_ite in range(1, len(arr_table.columns)+1):
    hit_sum = 0
    for i in range(0, len(arr_table)):
        true_dict = {}
        j = 0
        true_arr_values = arr_table.loc[i].tolist()
        for column in arr_table.columns.tolist():
            true_dict[column] = true_arr_values[j]
            j += 1
        true_dict = sorted(true_dict.items(), key=lambda x:x[1], reverse=True)


        pred_dict = {}
        j = 0
        pred_arr_values = ranking_table.loc[i].tolist()
        for column in ranking_table.columns.tolist():
            pred_dict[column] = pred_arr_values[j]
            j += 1
        pred_dict = sorted(pred_dict.items(), key=lambda x:x[1], reverse=True)

        hit = 0
        for pred_index in range(0, top_ite):
            for true_index in range(0, top_ite):
                if pred_dict[pred_index][0] == true_dict[true_index][0]:
                    hit = 1
                    break
            if hit == 1:
                hit_sum += 1
                break
            
    y_axis.append(hit_sum / len(arr_table))
    x_axis.append(top_ite)


top = 3
hit_rate = y_axis[top-1]
print('Hit Rate (Top '+str(top)+'): ' + str(hit_rate*100) + '%')

plt.ylabel('Hit Rate')
plt.xlabel('Top N')
plt.xticks(x_axis)
plt.bar(x_axis, y_axis)
plt.show()

## Recommendation Accuracy

In [ ]:
y_axis = []
x_axis = []

for top_ite in range(1, len(arr_table.columns)+1):
    rec_acc_sum = 0
    for i in range(0, len(arr_table)):
        true_dict = {}
        j = 0
        true_arr_values = arr_table.loc[i].tolist()
        for column in arr_table.columns.tolist():
            true_dict[column] = true_arr_values[j]
            j += 1
        true_dict = sorted(true_dict.items(), key=lambda x:x[1], reverse=True)


        pred_dict = {}
        j = 0
        pred_arr_values = ranking_table.loc[i].tolist()
        for column in ranking_table.columns.tolist():
            pred_dict[column] = pred_arr_values[j]
            j += 1
        pred_dict = sorted(pred_dict.items(), key=lambda x:x[1], reverse=True)

        i = 0
        true_set = set()
        for alg in true_dict:
            if i == top_ite:
                break
            true_set.add(alg[0])
            i += 1        

        i = 0
        pred_set = set()
        for alg in pred_dict:
            if i == top_ite:
                break
            pred_set.add(alg[0])
            i += 1
        
        intersection_set = true_set.intersection(pred_set)
        union_set = true_set.union(pred_set)

        rec_acc_sum += len(intersection_set) / len(union_set)

        # pdb.set_trace()

    y_axis.append(rec_acc_sum / len(arr_table))
    x_axis.append(top_ite)


top = 3
recommendation_accuracy = y_axis[top-1]
print('Recommendation Accuracy (Top '+str(top)+'): ' + str(recommendation_accuracy*100) + '%')

plt.ylabel('Recommendation Accuracy')
plt.xlabel('Top N')
plt.xticks(x_axis)
plt.bar(x_axis, y_axis)
plt.show()